In [2]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'cv2'

In [ ]:
img_array = cv2.imread("data/test/closed/s0001_00056_0_0_0_0_0_01.png", cv2.IMREAD_GRAYSCALE)

In [ ]:
plt.imshow(img_array, cmap="gray")
plt.show()
print(img_array.shape)

In [ ]:
Datadirectory = "data/train/"

Classes = ["closed", "open"]

for category in Classes:
    path = os.path.join(Datadirectory, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        backtorgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
        plt.imshow(img_array, cmap="gray")
        plt.show()
        break
    break

In [ ]:
img_size = 224
new_array = cv2.resize(backtorgb, (img_size, img_size))
plt.imshow(new_array, cmap="gray")
plt.show()

In [ ]:
training_Data = []

def create_training_Data():
    for category in Classes:
        path = os.path.join(Datadirectory, category)
        class_num = Classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                backtorgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
                new_array = cv2.resize(backtorgb, (img_size, img_size))
                training_Data.append([new_array, class_num])
            except Exception as e:
                pass

In [ ]:
create_training_Data()
print(len(training_Data))

In [ ]:
import random
random.shuffle(training_Data)

In [ ]:
X = []
y = []

for features, label in training_Data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, img_size, img_size, 3)


In [ ]:
print(X.shape)

In [ ]:
X = X/255.0

In [ ]:
Y = np.array(y)

In [ ]:
import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()


In [ ]:
#later to load again directly
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = tf.keras.applications.mobilenet.MobileNet()
model.summary()

In [ ]:
base_input = model.layers[0].input
base_output = model.layers[-4].output

In [ ]:
Flat_layer = layers.Flatten()(base_output)
final_output = layers.Dense(1)(Flat_layer)  # one node (1/0)
final_output = layers.Activation('sigmoid')(final_output)


In [ ]:
new_model = keras.Model(inputs=base_input, outputs=final_output)
new_model.summary()


In [ ]:
new_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
new_model.fit(X, Y, epochs=25, validation_split=0.1)  # training



In [ ]:
# Save the model
new_model.save('my_model_new.h5')

In [ ]:
# Load the model
new_model = tf.keras.models.load_model('my_model_new.h5')

In [ ]:
# Checking the network for predictions
img_array = cv2.imread("s0012_08255_0_0_1_1_0_02.png", cv2.IMREAD_GRAYSCALE)
backtorgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
new_array = cv2.resize(backtorgb, (img_size, img_size))

In [ ]:
# Prepare the input data for prediction
X_input = np.array(new_array).reshape(1, img_size, img_size, 3)
X_input = X_input / 255.0  # Normalize the input

In [ ]:
# Display the resized image
plt.imshow(new_array)


In [ ]:
# Predict using the loaded model
prediction = new_model.predict(X_input)
print(prediction)

In [ ]:
img = cv2.imread('sad_women.jpg')

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR3RGB))

In [ ]:
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'harcascade_frontalface_default.xml')

In [ ]:
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'harcascade_eye.xml')

In [ ]:
gray = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)

In [ ]:
eyes = eye_cascade.detectMultiScale(gray, 1.1,4)

In [ ]:
for(x, y, w, h) in eyes:
    cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR3RGB))

In [ ]:
# Load Haar cascade for eye detection
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Load the image
#img = cv2.imread('path_to_your_image.jpg')  # Replace 'path_to_your_image.jpg' with the actual image path

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect eyes in the grayscale image
eyes = eye_cascade.detectMultiScale(gray, 1.1, 4)

for x, y, w, h in eyes:
    # Crop the region of interest (eye region)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]

    # Detect eyes in the cropped eye region
    eyess = eye_cascade.detectMultiScale(roi_gray)

    if len(eyess) == 0:
        print("Eyes are not detected")
    else:
        for (ex, ey, ew, eh) in eyess:
            # Crop the detected eye
            eyes_roi = roi_color[ey:ey+eh, ex:ex+ew]

# Display the cropped eye region
plt.imshow(cv2.cvtColor(eyes_roi, cv2.COLOR_BGR2RGB))
plt.show()

# Check the shape of the cropped eye
print("Cropped Eye Shape:", eyes_roi.shape)

# Resize and preprocess the cropped eye for deep learning model input
final_image = cv2.resize(eyes_roi, (224, 224))
final_image = np.expand_dims(final_image, axis=0)  # Add batch dimension
final_image = final_image / 255.0  # Normalize pixel values

# Check the shape of the final preprocessed image
print("Preprocessed Image Shape:", final_image.shape)

# Make predictions using the pre-trained model
predictions = new_model.predict(final_image)
print("Predictions:", predictions)

In [ ]:
import cv2
import numpy as np

# Load Haar cascade for face detection
path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(path)

# Open the webcam
cap = cv2.VideoCapture(1)  # Use camera index 1, change it to 0 if needed

# Check if the webcam is opened correctly
if not cap.isOpened():
    cap = cv2.VideoCapture(0)  # Try camera index 0 if index 1 failed
    if not cap.isOpened():
        raise IOError("Cannot open webcam")

while True:
    ret, frame = cap.read()  # Read frame from the webcam
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

    # Convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    eyes = eye_cascade.detectMultiScale(gray, 1.1, 4)

    # Draw rectangles around the detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Display status (Open/Closed Eyes) on the frame
    font = cv2.FONT_HERSHEY_SIMPLEX
    status = "Open Eyes" if len(faces) > 0 else "Closed Eyes"
    cv2.putText(frame, status, (50, 50), font, 3, (0, 0, 255), 2, cv2.LINE_4)

    cv2.imshow('Drowsiness Detection Tutorial', frame)  # Display the frame

    if cv2.waitKey(2) & 0xFF == ord('q'):  # Press 'q' to quit
        break

cap.release()  # Release the webcam
cv2.destroyAllWindows()  # Close all windows
